### CPSC 672 Network Project

In [89]:
import numpy as np
import pandas as pd
import networkx as nx
import random
from random import sample
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
import ast
import csv
%matplotlib inline

### Data

The following few cells read in our data from a excel spreadsheet and organize them into a few arrays:
- stock_symbols contains all the actual stock names
- dates contains all the column headings (which are dates) from Feb 2010 to Dec 2021

In [38]:
df = pd.read_excel('data/all_data.xlsx', sheet_name='percentChange')

In [55]:
stock_symbols = []
for stock in df['Stock']:
    stock_symbols.append(stock)

dates = df.columns.values
dates = dates[2:]

### Calculating links

To get the links between any two stocks we have to obtain the correlation with the following formula:

                            corr(a,b) = min(a,b) / max(a,b)
the getLinks() function does just that by comparing every stock, with every other stock, for any input date (column)

In [84]:
def getLinks(date):
    links = []
    elems = len(df[date])
    for i in range(elems):
        for j in range(i+1,elems):
            correlation = min(df[date][i],df[date][j]) / max(df[date][i],df[date][j])
            links.append((stock_symbols[i],stock_symbols[j],correlation))
    return links

In [85]:
test = getLinks(dates[0])
print(test[0:100])

2935


/var/folders/3t/dt8n7hw17cg52mnvdkmzy3g40000gn/T/ipykernel_82054/301231491.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  correlation = min(df[date][i],df[date][j]) / max(df[date][i],df[date][j])
/var/folders/3t/dt8n7hw17cg52mnvdkmzy3g40000gn/T/ipykernel_82054/301231491.py:7: RuntimeWarning: invalid value encountered in double_scalars
  correlation = min(df[date][i],df[date][j]) / max(df[date][i],df[date][j])


### Exporting
At this point all the stock symbols are exported to a csv,
similarly the links for a particular date are also exported to a csv.

In [112]:
with open('data/stocks.csv','w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["ID	label"])
    for stock in stock_symbols:
        writer.writerow([str(stock_symbols.index(stock))+"\t"+stock])

In [115]:
with open('data/links.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Source	Target	Weight"])
    for link in test:
        if(link[2] > 0.80 and link[2] <= 1):
            writer.writerow([str(stock_symbols.index(link[0]))+"\t"+str(stock_symbols.index(link[1]))+"\t"+str(link[2])])